In [1]:
import sys, os
import numpy as np
import time
from sklearn import decomposition
import pandas as pd
import torch
import PIL

project_root = '/user_data/mmhender/featsynth/'
texture_synth_root = os.path.join(project_root, 'texture_synthesis')

# these are in the 'texture_synthesis' folder
sys.path.append(os.path.join(texture_synth_root, 'code'))
import utilities
import model_spatial

# from image_analysis import extract_resnet_features
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [4]:
image_set_name = 'images_ecoset64'
folder_images = os.path.join(project_root, 'features','raw')

In [8]:
image_set_name = 'images_things200'
folder_images = os.path.join(project_root, 'features','raw')

In [9]:
list_orig = os.path.join(folder_images, '%s_list.csv'%(image_set_name))
list_new = os.path.join(folder_images, '%s_grayscale_list.csv'%(image_set_name))
print('duplicating list from %s to %s'%(list_orig, list_new))
d = pd.read_csv(list_orig)
d.to_csv(list_new)


duplicating list from /user_data/mmhender/featsynth/features/raw/images_things200_list.csv to /user_data/mmhender/featsynth/features/raw/images_things200_grayscale_list.csv


In [2]:
image_set_name = 'images_ecoset'
layer_process = 'pool1'
debug = False
n_per_categ = 100
n_cv = 10

In [3]:
print('debug=%s'%debug)

# load image features
feat_path = os.path.join(project_root, 'features', 'gram_matrices')

feat_all = []

layers_process = [layer_process]
print(layers_process)
for li in range(len(layers_process)):

    feat_file_name = os.path.join(feat_path, \
                                  '%s_gram_matrices_%s_pca.npy'%(image_set_name,\
                                                           layers_process[li]))
    print(feat_file_name)
    feat = np.load(feat_file_name)

    feat_all += [feat]

feat_all = np.concatenate(feat_all, axis=1)
feat = feat_all
feat = scipy.stats.zscore(feat, axis=0)

save_dir = os.path.join(feat_path, 'categ_discrim')
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

fn2save = os.path.join(save_dir, 'categ_discrim_allbasic_%s_%s_%dpercateg.npy'%\
                       (image_set_name, layer_process, n_per_categ))
print('will save to %s'%fn2save)


# load corresponding labels for the images
image_list_filename = os.path.join(project_root, 'features','raw', '%s_list.csv'%(image_set_name))
labels = pd.read_csv(image_list_filename)
n_images = labels.shape[0]

# figure out some image/category properties here
n_ims_each = np.sum(np.array(labels['basic_name'])==np.array(labels['basic_name'])[0])
basic_names = np.array(labels['basic_name'][0::n_ims_each])
super_names_long = np.array(labels['super_name'][0::n_ims_each])

n_basic = len(basic_names)
n_super = len(np.unique(super_names_long))
n_basic_each_super  = int(n_basic/n_super)
super_names = super_names_long[0::n_basic_each_super]

# info about ecoset categories
fn = os.path.join(ecoset_info_path, 'categ_use_ecoset.npy')
info = np.load(fn, allow_pickle=True).item()

super_acc = np.zeros((1,))
super_dprime = np.zeros((1,))
super_acc_each_supcat = np.zeros((n_super,))
super_dprime_each_supcat = np.zeros((n_super,))
    
    
basic_acc = np.zeros((n_super,))
basic_dprime = np.zeros((n_super,))
basic_acc_each_bascat = np.zeros((n_basic,))
basic_dprime_each_bascat = np.zeros((n_basic,))

# create a set of images that have the desired number per superord categ
ims_use_subsample = np.zeros((n_images,),dtype=bool)

# n_per_categ is how many we want total per superordinate categ.
# want to divide these evenly across the basics
n_per_basic = n_per_categ

for bname in basic_names:

    inds = np.where((np.array(labels['basic_name'])==bname))[0]
    print('there are %d images for %s, choosing %d'%(len(inds), bname, n_per_basic))
    inds_use = np.random.choice(inds, n_per_basic, replace=False)

    ims_use_subsample[inds_use] = True  

print('total ims subsampled for basic-all: %d'%(np.sum(ims_use_subsample)))

debug=False
['pool1']
/user_data/mmhender/featsynth/features/gram_matrices/images_ecoset_gram_matrices_pool1_pca.npy
will save to /user_data/mmhender/featsynth/features/gram_matrices/categ_discrim/categ_discrim_allbasic_images_ecoset_pool1_100percateg.npy
there are 500 images for beetle, choosing 100
there are 500 images for bee, choosing 100
there are 500 images for butterfly, choosing 100
there are 500 images for grasshopper, choosing 100
there are 500 images for caterpillar, choosing 100
there are 500 images for ant, choosing 100
there are 500 images for moth, choosing 100
there are 500 images for mosquito, choosing 100
there are 500 images for pea, choosing 100
there are 500 images for corn, choosing 100
there are 500 images for pumpkin, choosing 100
there are 500 images for onion, choosing 100
there are 500 images for cabbage, choosing 100
there are 500 images for lettuce, choosing 100
there are 500 images for beet, choosing 100
there are 500 images for asparagus, choosing 100
the

In [26]:
super_acc = np.zeros((1,))
super_dprime = np.zeros((1,))
super_acc_each_supcat = np.zeros((n_super,))
super_dprime_each_supcat = np.zeros((n_super,))


In [18]:
)

In [6]:
# do basic-all classification
inds = ims_use_subsample

f = feat[inds,:]

print('size of f:')
print(f.shape)

labs_use = np.array(labels['basic_index'])[inds]

sys.stdout.flush()
pred_labs = logreg_clf(f, labs_use, cv_labs=None, n_cv=n_cv, debug=debug).astype(int)

size of f:
(6400, 500)
(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), array([640, 640, 640, 640, 640, 640, 640, 640, 640, 640]))
cv 0: best C is 0.01000000
cv 1: best C is 0.01000000


/home/mmhender/imstat_env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


cv 2: best C is 0.01000000
cv 3: best C is 0.01000000


/home/mmhender/imstat_env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


cv 4: best C is 0.01000000
cv 5: best C is 0.01000000
cv 6: best C is 0.01000000
cv 7: best C is 0.01000000
cv 8: best C is 0.01000000
cv 9: best C is 0.01000000


In [27]:
# mapping the basic-level labels into superordinate categs
super_inds_long = np.repeat(np.arange(n_super), n_basic_each_super)
                            
pred_labs_super = super_inds_long[pred_labs]
actual_labs_super = super_inds_long[labs_use]

a = np.mean(pred_labs_super==actual_labs_super)
super_acc[0] = a
d = stats_utils.get_dprime(pred_labs_super, actual_labs_super)
super_dprime[0] = d
print('super acc=%.2f, super dprime=%.2f'%(a, d))
sys.stdout.flush()

super acc=0.47, super dprime=1.37


In [30]:
# get accuracy for each individual super-category
for si in np.unique(actual_labs_super):

    # accuracy just for images in this category
    inds = actual_labs_super==si
    assert(np.sum(inds)==f.shape[0]/n_super)
    super_acc_each_supcat[si] = np.mean(pred_labs_super[inds]==actual_labs_super[inds])

    # getting d-prime for each category
    # this is actually using all trials - but only measuring performance 
    # based on whether the presence/absence of this categ was correct.
    # convert the labels to binary yes/no
    pred_tmp = (pred_labs_super==si).astype('int')
    labs_tmp = (actual_labs_super==si).astype('int')

    super_dprime_each_supcat[si] = stats_utils.get_dprime(pred_tmp, labs_tmp)


In [33]:
np.mean(super_dprime_each_supcat)

1.3668467530410147

In [34]:
np.mean(super_acc_each_supcat)

0.47203124999999996

In [21]:
np.unique(pred_labs_super, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7]),
 array([738, 790, 735, 889, 679, 773, 831, 965]))

In [23]:
np.unique(actual_labs_super, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7]),
 array([800, 800, 800, 800, 800, 800, 800, 800]))

In [7]:
assert(not np.any(np.isnan(pred_labs)))
a = np.mean(pred_labs==labs_use)
basic_acc[0] = a
d = stats_utils.get_dprime(pred_labs, labs_use)
basic_dprime[0] = d
print('basic acc=%.2f, basic dprime=%.2f'%(a, d))
sys.stdout.flush()

basic acc=0.27, basic dprime=1.62


In [8]:
np.unique(pred_labs, return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
        51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63]),
 array([ 71,  82,  95,  89, 101,  88,  80, 132,  91,  88, 102, 106, 102,
        110,  88, 103,  93,  78,  81, 100, 114, 100, 104,  65,  77, 106,
        125,  99, 124,  96, 123, 139,  78,  83,  83,  93,  85,  82,  87,
         88, 117,  92,  98,  96,  98,  79, 104,  89, 122,  84,  90, 114,
        119,  86, 149,  67, 150, 111, 144, 117, 108, 104, 115, 116]))

In [9]:
np.unique(labs_use, return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
        51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63]),
 array([100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100,
        100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100,
        100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100,
        100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100,
        100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100]))

In [11]:
# get accuracy for each individual basic-category
for bi in np.unique(labs_use):

    # accuracy just for images in this category
    inds = labs_use==bi
    assert(np.sum(inds)==f.shape[0]/n_basic)
    basic_acc_each_bascat[bi] = np.mean(pred_labs[inds]==labs_use[inds])

    # getting d-prime for each category
    # this is actually using all trials - but only measuring performance 
    # based on whether the presence/absence of this categ was correct.
    # convert the labels to binary yes/no
    pred_tmp = (pred_labs==bi).astype('int')
    labs_tmp = (labs_use==bi).astype('int')

    basic_dprime_each_bascat[bi] = stats_utils.get_dprime(pred_tmp, labs_tmp)

    

In [12]:
basic_acc_each_bascat

array([0.07, 0.19, 0.36, 0.11, 0.14, 0.18, 0.08, 0.39, 0.34, 0.43, 0.42,
       0.33, 0.34, 0.53, 0.15, 0.27, 0.28, 0.35, 0.38, 0.27, 0.38, 0.38,
       0.32, 0.42, 0.26, 0.33, 0.2 , 0.28, 0.28, 0.13, 0.37, 0.47, 0.12,
       0.09, 0.05, 0.14, 0.12, 0.13, 0.14, 0.1 , 0.25, 0.14, 0.38, 0.18,
       0.4 , 0.23, 0.29, 0.12, 0.25, 0.15, 0.16, 0.47, 0.17, 0.14, 0.54,
       0.14, 0.64, 0.41, 0.53, 0.27, 0.24, 0.48, 0.34, 0.31])

In [14]:
np.mean(basic_acc_each_bascat)

0.27421874999999996

In [13]:
basic_dprime_each_bascat

array([0.84464207, 1.44845158, 1.99239963, 1.0185689 , 1.12233604,
       1.37118286, 0.87074247, 1.89710177, 1.95120041, 2.2736235 ,
       2.14270423, 1.83063073, 1.88508885, 2.4389334 , 1.23411051,
       1.6422881 , 1.73175689, 2.08100316, 2.16084284, 1.6577309 ,
       1.9496203 , 2.02686439, 1.80811523, 2.48087338, 1.76124782,
       1.83063073, 1.286424  , 1.69830663, 1.58100423, 1.09463478,
       1.87532521, 2.10542019, 1.13385466, 0.92458105, 0.60024339,
       1.15985872, 1.0955571 , 1.16562414, 1.19022455, 0.96354546,
       1.5062003 , 1.16477768, 2.03911692, 1.32973193, 2.10386557,
       1.63136745, 1.70680427, 1.07508412, 1.4852393 , 1.25558188,
       1.2708782 , 2.22789016, 1.18551084, 1.19549469, 2.26843315,
       1.3101818 , 2.56563735, 2.05900297, 2.26026929, 1.57653676,
       1.51006022, 2.32006071, 1.8180366 , 1.715897  ])

In [15]:
np.mean(basic_dprime_each_bascat)

1.6235774057067438

In [ ]:
basic_dprime_each_bascat

In [5]:

    
def logreg_clf(feat, labs, cv_labs=None, n_cv = 10, debug=False):

    
    if cv_labs is None:
        # making random cross-validation labels
        # balance classes as closely as possible
        cv_labs = np.zeros_like(labs)
        unvals = np.unique(labs)
        for uu in unvals:
            inds = np.where(labs==uu)[0]
            cv_tmp = np.tile(np.arange(n_cv), [int(np.ceil(len(inds)/n_cv)),])
            cv_tmp = cv_tmp[np.random.permutation(len(cv_tmp))][0:len(inds)]
            cv_labs[inds] = cv_tmp
    
    pred_labs = np.full(fill_value=np.nan, shape=cv_labs.shape)
    
    print(np.unique(cv_labs, return_counts=True))
    
    for cvi, cv in enumerate(np.unique(cv_labs)):

        if debug and cvi>1:
            continue
            
        trninds = cv_labs!=cv
        tstinds = cv_labs==cv

        cs = np.logspace(-10, 0, 16)
        try:
            clf = sklearn.linear_model.LogisticRegressionCV(multi_class='multinomial', \
                                                          Cs = cs, fit_intercept=True, \
                                                          penalty = 'l2', \
                                                          refit=True, \
                                                          max_iter=10000, \
                                                         )
            clf.fit(feat[trninds,:], labs[trninds])
            print('cv %d: best C is %.8f'%(cvi, clf.C_[0]))
            sys.stdout.flush()

            p = clf.predict(feat[tstinds,:])

            pred_labs[tstinds] = p
            
        except:
            print('WARNING: problem with classifer, returning nans')
            pred_labs[tstinds] = np.nan
            
    return pred_labs




